# Hikma COVID-19 Response State Policies

License: Apache 2.0

In [0]:
import copy
import urllib
import json

import pandas as pd
import numpy as np


## Read the data from CSV from the Kaiser Family Foundation

In [0]:
# Before running this step, download the exported CSV from here for the table titled "State Actions to Mitigate the Spread of COVID-19", and then upload it to the colab.
# raw_policies = pd.read_csv('raw_data.csv',skiprows=[0,1])

raw_policies = pd.read_csv('https://raw.githubusercontent.com/hikmahealth/covid19countymap/master/KFF_state_data.csv',skiprows=[0,1])

## Clean up the state-level data

### Encode free-text column values

In [0]:
columns = ['location', 'home', 'travel', 'business', 'gathering', 'school', 'restaurant', 'election', 'emergency', 'footnote']

def column_renamer(column_name):
  for c in columns:
    if c in column_name.lower():
      if c == 'business':
        return 'work'
      if c == 'gathering':
        return 'event' 
      if c == 'home':
        return 'shelter'
      else:
        return c
  return column_name

state_policies = raw_policies.rename(columns=column_renamer)

# remove extraneous rows that are not states
state_policies = state_policies.drop([0])
state_policies = state_policies.drop(range(52,65))

# remove unnecessary columns
state_policies = state_policies.drop(['restaurant', 'election', 'emergency','footnote'], axis = 1)

# Texas 'Other' stay-at-home order is a 'Required Yes' as verified by NYT as of April 2
# https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html
state_policies.iloc[43,1] = 'Statewide'

# New Mexico 'All Air Travelers' travel restriction is now a 'Required Yes' as of April 11
# https://www.nytimes.com/2020/04/10/travel/coronavirus-us-travel-driving-restrictions.html
state_policies.iloc[31,2] = 'All Travelers'


In [0]:
def make_dict(entries):
  mapping = {v: i for i, v in enumerate(entries)}
  mapping[""] = 0  # Map no response to "do not know"
  return mapping

# Encoders are constructed such that 0 = no info; 1 = no; 2 = recommended yes; 3 = required yes
travel_encoder = make_dict(['Other', '-', 'From Certain States',
                            'All Travelers'])
zero_encoder = make_dict(['none'])
event_encoder = make_dict(['Other', '-', '>10 People Prohibited',
                            'All Gatherings Prohibited'])
school_encoder = make_dict(['Other', '-',
                                 'Effectively Closed',
                                 'Yes'])
work_encoder = make_dict(['Other', '-',
                         'All Non-Essential Retail Businesses', 'All Non-Essential Businesses'])
shelter_encoder = make_dict(['Other', '-', 'High-risk Groups',
                            'Statewide']) 

# No states have blanket public transportation shutdowns as of 2020-04-11.
# https://www.nytimes.com/2020/04/10/travel/coronavirus-us-travel-driving-restrictions.html
state_policies["transport"] = False
# All states have at least some active testing as of 2020-04-11
# https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/testing-in-us.html
state_policies["testing"] = True

# Initialize all shelter in place as yes since most are yes
# state_policies["shelter"] = 'Yes'


In [0]:
encoders = {
    'travel': travel_encoder,
    'work': work_encoder,
    'event': event_encoder,
    'school': school_encoder,
    'shelter': shelter_encoder
}

for column, encoder in encoders.items():
  state_policies[column] = state_policies[column].map(encoder)

In [6]:
# Add all NO SHELTER states from NYT data (current as of 4/7/20)
# No longer needed, working off KFF data 4/11/20 -SE
# https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html
# state_policies.iloc[3,7] = 1 #Arkansas
# state_policies.iloc[34,7] = 1 #North Dakota
# state_policies.iloc[41,7] = 1 #South Dakota

state_policies.head()

,location,shelter,travel,work,event,school,transport,testing
1,Alabama,3,1,3,2,3,False,True
2,Alaska,3,3,3,3,3,False,True
3,Arizona,3,2,3,2,3,False,True
4,Arkansas,1,1,1,2,3,False,True
5,California,3,1,3,3,2,False,True


In [0]:
# Casting all as int
state_policies['travel'] = state_policies['travel'].astype('int')
state_policies['event'] = state_policies['event'].astype('int')
state_policies['shelter'] = state_policies['shelter'].astype('int')

In [0]:
with urllib.request.urlopen("https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv") as infile:
  state_fips = pd.read_csv(infile)

In [0]:
state_fips.set_index("stname", inplace=True)

In [0]:
state_policies['location'] = state_policies['location'].map(state_fips[" st"])

In [0]:
state_policies.set_index('location', inplace=True)

In [12]:
state_policies.head()

,shelter,travel,work,event,school,transport,testing
location,,,,,,,
1,3,1,3,2,3,False,True
2,3,3,3,3,3,False,True
4,3,2,3,2,3,False,True
5,1,1,1,2,3,False,True
6,3,1,3,3,2,False,True


### Data Export

In [0]:
with open("state_policies.json", "w") as outfile:
  outfile.write(state_policies.to_json(orient="index"))

In [0]:
# Dump column decoders as well.
# We reverse the map from integer to string, but not no information default.
decoders = {c: {i: s for s, i in enc.items() if i}
            for c, enc in encoders.items()}

In [0]:
with open("policy_decoders.json", "w") as outfile:
  json.dump(decoders, outfile)